In [2]:
!pip install ultralytics 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found exi

In [3]:
!pip install pyquaternion 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nuscenes-devkit 1.1.11 requires descartes, which is not installed.
nuscenes-devkit 1.1.11 requires fire, which is not installed.
nuscenes-devkit 1.1.11 requires matplotlib<3.6.0, but you have matplotlib 3.7.2 which is incompatible.
nuscenes-devkit 1.1.11 requires Shapely<2.0.0, but you have shapely 2.1.0 which is incompatible.


In [ ]:
!pip install --no-deps nuscenes-devkit

In [9]:
!tar -xzf /kaggle/input/3d-cv-hw/v1.0-mini.tgz -C /kaggle/working/

In [10]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import imageio
from ultralytics import YOLO
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud
from pyquaternion import Quaternion
from sklearn.decomposition import PCA

data_root = '/kaggle/working/'
nusc = NuScenes(version='v1.0-mini', dataroot=data_root, verbose=False)

In [11]:
yolo_model = YOLO('yolov8n.pt')
my_scene = nusc.scene[0]

100%|██████████| 6.25M/6.25M [00:00<00:00, 70.0MB/s]


In [12]:
def project_lidar_to_camera(nusc, sample_token, cam_channel='CAM_FRONT'):
    sample = nusc.get('sample', sample_token)
    
    cam_token = sample['data'][cam_channel]
    lidar_token = sample['data']['LIDAR_TOP']
    
    cam_data = nusc.get('sample_data', cam_token)
    lidar_data = nusc.get('sample_data', lidar_token)

    lidar_filepath = os.path.join(nusc.dataroot, lidar_data['filename'])
    pc = LidarPointCloud.from_file(lidar_filepath)
    
    lidar_calib = nusc.get('calibrated_sensor', lidar_data['calibrated_sensor_token'])
    lidar_to_ego_rot = Quaternion(lidar_calib['rotation']).rotation_matrix
    lidar_to_ego_trans = np.array(lidar_calib['translation'])
    
    pc.rotate(lidar_to_ego_rot)
    pc.translate(lidar_to_ego_trans)
    
    lidar_points_in_ego_frame = pc.points[:3, :]
    
    cam_calib = nusc.get('calibrated_sensor', cam_data['calibrated_sensor_token'])
    ego_to_cam_rot = Quaternion(cam_calib['rotation']).inverse.rotation_matrix
    ego_to_cam_trans = -np.array(cam_calib['translation']) @ ego_to_cam_rot.T

    pc.rotate(ego_to_cam_rot)
    pc.translate(ego_to_cam_trans)

    camera_intrinsic = np.array(cam_calib['camera_intrinsic'])
    
    depths = pc.points[2, :]
    points_in_front_of_camera = depths > 0
    
    points_3d_in_camera_frame = pc.points[:3, points_in_front_of_camera]
    points_depth = depths[points_in_front_of_camera]
    
    points_2d_projected = camera_intrinsic @ points_3d_in_camera_frame
    
    points_2d = points_2d_projected[:2, :] / points_2d_projected[2, :]
    
    img_h, img_w = cam_data['height'], cam_data['width']
    inside_image_mask = (points_2d[0, :] >= 0) & (points_2d[0, :] < img_w) & \
                        (points_2d[1, :] >= 0) & (points_2d[1, :] < img_h)
                        
    final_points_2d = points_2d[:, inside_image_mask].T
    final_points_depth = points_depth[inside_image_mask]
    
    lidar_points_in_ego_frame_filtered = lidar_points_in_ego_frame[:, points_in_front_of_camera][:, inside_image_mask]
    
    image_path = os.path.join(nusc.dataroot, cam_data['filename'])
    
    return final_points_2d, final_points_depth, image_path, lidar_points_in_ego_frame_filtered

In [13]:
output_dir = '/kaggle/working/demo_frames/'
os.makedirs(output_dir, exist_ok=True)

current_sample_token = my_scene['first_sample_token']
frame_count = 0

while current_sample_token:
    sample = nusc.get('sample', current_sample_token)
    cam_front_token = sample['data']['CAM_FRONT']
    cam_front_data = nusc.get('sample_data', cam_front_token)
    image_path = os.path.join(nusc.dataroot, cam_front_data['filename'])
    
    results = yolo_model(image_path, verbose=False)
    bboxes_2d = results[0].boxes.xyxy.cpu().numpy()

    points_2d, depths, image_path, lidar_points_ego = project_lidar_to_camera(nusc, current_sample_token)

    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    for i, bbox in enumerate(bboxes_2d):
        x1, y1, x2, y2 = bbox
        
        mask_inside_box = (points_2d[:, 0] > x1) & (points_2d[:, 0] < x2) & \
                          (points_2d[:, 1] > y1) & (points_2d[:, 1] < y2)
        
        lidar_points_for_box_ego = lidar_points_ego[:, mask_inside_box]
        
        if lidar_points_for_box_ego.shape[1] > 0:
            distances = np.linalg.norm(lidar_points_for_box_ego, axis=0)
            median_distance = np.median(distances)
            
            points_to_draw = points_2d[mask_inside_box]
            for point in points_to_draw:
                cv2.circle(image, (int(point[0]), int(point[1])), 2, (255, 0, 255), -1)

            ix1, iy1, ix2, iy2 = map(int, bbox)
            cv2.rectangle(image, (ix1, iy1), (ix2, iy2), (0, 255, 0), 2)
            label = f"{median_distance:.2f} m"
            cv2.putText(image, label, (ix1, iy1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    output_frame_path = os.path.join(output_dir, f"frame_{frame_count:04d}.png")
    plt.imsave(output_frame_path, image)

    current_sample_token = sample['next']
    frame_count += 1
    
    if frame_count > 40: 
        break

In [15]:
images = []
for i in range(frame_count):
    filename = os.path.join(output_dir, f"frame_{i:04d}.png")
    if os.path.exists(filename):
        images.append(imageio.imread(filename))

if images:
    imageio.mimsave('/kaggle/working/driver_assistance_demo.gif', images, fps=20)
    print("GIF создан: /kaggle/working/driver_assistance_demo.gif")
else:
    print("Кадры для GIF не найдены.")

/tmp/ipykernel_35/2984815143.py:5: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(filename))


Демонстрационный GIF-файл успешно создан: /kaggle/working/driver_assistance_demo.gif
